In [1]:
import numpy as np 
from numba import cuda
import matplotlib.pyplot as plt
import math
from time import time

In [19]:
DELTA_T = 0.001
EPOCH = 1000
TPB = 32
@cuda.jit
def ode_kernel_c(d_f):
    i = cuda.grid(1)
    n = d_f.shape[0]
    if i < n:
        for e in range(EPOCH):
            x = d_f[i][0]
            v = d_f[i][1]
            d_f[i][0] = x + v * DELTA_T
            d_f[i][1] = v - x * DELTA_T

In [20]:
def ode_solver():
    N = 100
    scale = np.linspace(0, 1, N)
    f = np.empty(N * 2).reshape((N, 2))
    for i in range(N):
        f[i][0] = 3 * math.cos(2*np.pi*scale[i])
        f[i][1] = 3 * math.sin(2*np.pi*scale[i])
#     for item in f:
#         print(item[0]**2+item[1]**2)
    d_f = cuda.to_device(f)
    gridDims = (N + TPB - 1) // TPB
    blockDims = TPB
    ode_kernel_c[gridDims, blockDims](d_f)
    f = d_f.copy_to_host()
    return f

In [21]:
f = ode_solver()
g = []
for item in f:
    g.append(item[0]**2+item[1]**2)
g

[9.0090044969959031,
 9.0090044969958374,
 9.0090044969958623,
 9.0090044969958765,
 9.0090044969958676,
 9.0090044969958747,
 9.0090044969958392,
 9.0090044969958836,
 9.0090044969958996,
 9.0090044969958623,
 9.0090044969959102,
 9.0090044969958658,
 9.0090044969958605,
 9.0090044969958853,
 9.0090044969958569,
 9.0090044969958925,
 9.0090044969958747,
 9.0090044969958498,
 9.0090044969958782,
 9.0090044969958853,
 9.0090044969958321,
 9.0090044969958942,
 9.0090044969959262,
 9.0090044969958782,
 9.0090044969958853,
 9.0090044969958907,
 9.0090044969958534,
 9.0090044969958534,
 9.0090044969958978,
 9.0090044969959031,
 9.0090044969958836,
 9.00900449699588,
 9.0090044969958392,
 9.0090044969958747,
 9.0090044969958623,
 9.0090044969959031,
 9.009004496995864,
 9.0090044969958587,
 9.0090044969958818,
 9.0090044969958978,
 9.0090044969958711,
 9.0090044969958996,
 9.0090044969958996,
 9.0090044969958765,
 9.0090044969958889,
 9.0090044969958267,
 9.0090044969958925,
 9.0090044969958